In [6]:
!pip install yfinance
!pip install pytickersymbols

In [7]:
import yfinance
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
from pytickersymbols import PyTickerSymbols
from google.colab import files
from datetime import datetime, timedelta
from tqdm.notebook import tqdm
import random
import plotly.graph_objects as go

In [ ]:
start="2001-01-01"
end="2021-01-01"

period = 20
multiplier = 2

limit_inf = [.02]
#limit_inf = [.02,.02,.02, .02, .02]
limit_sup = [.04]
#limit_sup = [.04,.04,.04, .05, .05]
risk = [3]
#risk = [3,3,3,3,3]

precio_min = [20]
#precio_min = [20,20,25,25, 20]
precio_max = [35]
#precio_max = [35,40,40,40,35]

In [ ]:
ticker = yfinance.Ticker('QQQ')
df = ticker.history(interval="1d",start=start,end=end)
# df_to_plot = pd.DataFrame(index=df.index)

In [ ]:
stock_data = PyTickerSymbols()

nasdaq_100=pd.DataFrame(stock_data.get_stocks_by_index('NASDAQ 100'))
nasdaq_100 = nasdaq_100[nasdaq_100.symbol != 'CTRP'].reset_index().symbol.to_list()

sp_500=pd.DataFrame(stock_data.get_stocks_by_index('S&P 500'))
sp_500 = sp_500[sp_500.symbol != 'SSE']
sp_500 = sp_500[sp_500.symbol != 'RTN']
sp_500 = sp_500[sp_500.symbol != 'CXO']
sp_500 = sp_500[sp_500.symbol != 'JEC']
sp_500 = sp_500[sp_500.symbol != 'TIF'].reset_index().symbol.to_list()

total_tickers = nasdaq_100 + sp_500

today = datetime.today().strftime('%Y-%m-%d')

In [ ]:
len(total_tickers)

596

In [ ]:
#FILTRO PARA TICKERS

tickers_list = []
for i in range(len(total_tickers)):
  ticker = yfinance.download(total_tickers[i], today)[['Close', 'Volume']]
  if ticker['Close'][0] > 30 and ticker['Close'][0] < 120 and ticker['Volume'][0] > 5000000:
    tickers_list.append(total_tickers[i])

In [ ]:
tickers_list

In [ ]:
#COPIADOS DEL FILTRO EL 27/01/2021
tickers_list = ['CSCO','INTC','ATVI','CMCSA','EBAY','GILD','JD','MU','MDLZ','SBUX','AMD','KDP','CVX','CSCO','KO','DOW','XOM','INTC','MRK','PFE','VZ',
 'ATVI','CMCSA','EBAY','GILD','MU','MDLZ','SBUX','ABBV','MO','AIG','BAC','BMY','C','COP','CVS','GE','GM','MET','MS','NEE','ORCL','USB','WFC','AMD','BSX',
 'DAL','FITB','FCX','GIS','IPG','KR','MGM','NEM','LUV','TJX','TWTR','UAL','VFC','WYNN','CARR','DOW','LVS','VIAC','PENN']

len(tickers_list)

65

In [ ]:
#Eliminar tickers repetidos con "set" 

tickers_list = list(set(tickers_list))
len(tickers_list)

54

In [ ]:
# get stock info
#yfinance.Ticker("msft").info

In [ ]:
iteracion = pd.DataFrame(['Límite Inf', 'Límite Sup', 'Riesgo (R)', 'Precio min', 'Precio max'], 
                         index=['Límite Inf', 'Límite Sup', 'Riesgo (R)', 'Precio min', 'Precio max']).drop(columns=0)

In [ ]:
totales = pd.DataFrame(['Total de trades:', 'Trades/día', '% Winners', '% Greens', '% Reds', '% Losers', 'R/Green Trade', 'R/Red Trade', 'R/Trade'], 
                       index= ['Total de trades:','Trades/día', '% Winners','% Greens','% Reds','% Losers','R/Green Trade','R/Red Trade','R/Trade']).drop(columns=0)

In [ ]:
#%%time

#tickers_list = ['TSLA'] #PARA PRUEBA CON UN SOLO TICKER

for k in range(len(risk)):
  df_totales = pd.DataFrame(['Total Trades', 'Cerró', 'Green', 'Red', 'Stopped', 'Green PL (Rs)','Red PL (Rs)', 'Total PL'], 
                          index=['Total Trades', 'Cerró', 'Green', 'Red', 'Stopped','Green PL (Rs)','Red PL (Rs)', 'Total PL'])
  df_totales2 = pd.DataFrame.copy(df_totales.drop(columns=0))
  d = pd.DataFrame.copy(pd.DataFrame(index=df.index))

  for j in range(len(tickers_list)):

      ticker = yfinance.Ticker(tickers_list[j])
      df = ticker.history(interval="1d",start=start,end=end)
      #df['UpperBand'] = df['Close'].rolling(period).mean() + df['Close'].rolling(period).std() * multiplier
      #df['LowerBand'] = df['Close'].rolling(period).mean() - df['Close'].rolling(period).std() * multiplier

      df2 = df[['Open', 'High', 'Low', 'Close']]#, 'UpperBand']]#, 'LowerBand'
      df2 = df2.dropna()

      df2['%'] = (df2['Open'] - df2['Close'].shift(1)) / df2['Close'].shift(1)
      df2['Range'],df2['Cerró'],df2['Stopped'],df2['Red'],df2['Green'],df2['PL Red'],df2['PL Green'] = np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan
      
      for i in range(1,len(df2)):
        if (df2['%'][i] >= limit_inf[k] and df2['%'][i] < limit_sup[k] and df2['Open'][i] >= precio_min[k] and df2['Open'][i] < precio_max[k]):
          df2['Range'][i] = True

      for i in range(1,len(df2)):
        if df2['Range'][i] == True:
          stop = (df2['Open'][i]+((df2['Open'][i]-df2['Close'][i-1])/risk[k]))

          if df2['Low'][i] <= df['Close'][i-1]:
            df2['Cerró'][i] = 1
            df2['PL Green'][i] = risk[k]
            df2['Red'][i] = 0
            df2['Green'][i] = 0
          else:
            df2['Cerró'][i] = 0
            df2['PL Green'][i] = 0

          if df2['High'][i] >= stop:
            df2['Stopped'][i] = 1
            df2['PL Red'][i] = -1
            df2['Red'][i] = 0
            df2['Green'][i] = 0
          else:
            df2['Stopped'][i] = 0
            df2['PL Red'][i] = 0

      for i in range(1,len(df2)):
        if df2['Range'][i] == True:
          if df2['Stopped'][i] != True and df2['Cerró'][i] != True and df2['Close'][i] > df2['Open'][i] and df2['Open'][i] != df2['Close'][i-1]:
            df2['Red'][i] = 1
            df2['PL Red'][i] = (df2['Open'][i] - df2['Close'][i]) / ((df2['Open'][i]-df2['Close'][i-1])/risk[k])
            df2['PL Green'][i] = 0
            df2['Green'][i] = 0
          elif df2['Stopped'][i] != True and df2['Cerró'][i] != True and df2['Close'][i] < df2['Open'][i] and df2['Open'][i] != df2['Close'][i-1]:
            df2['Green'][i] = 1
            df2['PL Green'][i] = (df2['Open'][i] - df2['Close'][i]) / ((df2['Open'][i]-df2['Close'][i-1])/risk[k])
            df2['PL Red'][i] = 0
            df2['Red'][i] = 0
      
      df2['Total Trades'] = df2['Cerró'] + df2['Stopped'] + df2['Red'] + df2['Green']
      df2['PL'] = df2['PL Red'] + df2['PL Green']

      df_totales2.insert(j, tickers_list[j], [df2['Total Trades'].sum(), df2['Cerró'].sum(), df2['Green'].sum(),
                                           df2['Red'].sum(), df2['Stopped'].sum(), df2['PL Green'].sum(),
                                           df2['PL Red'].sum(), (df2['PL Green'].sum() + df2['PL Red'].sum())])
      
      pl_list = df2['PL'].to_list()

      while len(pl_list) < len(d):
        pl_list.append(0)

      d.insert(j, tickers_list[j], pl_list)

  df_totales2['sum'] = df_totales2.sum(axis=1)
  iteracion.insert(k,k, [limit_inf[k], limit_sup[k], risk[k], precio_min[k], precio_max[k]])
  #df_totales2 = df_totales.transpose()
  totales.insert(k, k, [df_totales2['sum'][0], df_totales2['sum'][0]/5040,df_totales2['sum'][1]/df_totales2['sum'][0], df_totales2['sum'][2]/df_totales2['sum'][0], 
                        df_totales2['sum'][3]/df_totales2['sum'][0], df_totales2['sum'][4]/df_totales2['sum'][0], 
                        df_totales2['sum'][5]/(df_totales2['sum'][1]+df_totales2['sum'][2]), df_totales2['sum'][6]/(df_totales2['sum'][3]+df_totales2['sum'][4]),
                        df_totales2['sum'][7]/df_totales2['sum'][0]])

In [ ]:
totales

In [ ]:
d['sum'] = d.sum(axis=1).cumsum()

In [ ]:
d

In [ ]:
plt.figure(figsize=[24,8])
d['sum']['2020-01-01':'2020-12-31'].plot()
plt.show()
#['2020-01-01':'2020-12-31']

In [ ]:
#df2[(df2['Range']==1.0)].head(65)
#df2[(df2['Trigger']==1.0) | (df2['Outside']==1.0)].head(60)
#df2.loc['2015-04-29':'2015-05-01']

In [ ]:
#df2.loc['2020-06-11':'2020-06-12']

# VERSION DEL 11/05/2021: ANALIZANDO STOCKS CUANDO SPY HACE GAP UP/DOWN MAYOR A 0.5%

In [ ]:
data_spy = yfinance.download('SPY', period='1Y')[['Open', 'High', 'Low', 'Close']]
gap = (data_spy['Open'] - data_spy['Close'].shift()) / data_spy['Close'].shift()
data_spy['Gap'] = gap
data_spy['Green_day'] = np.nan
data_spy['Red_day'] = np.nan
data_spy['Streak'] = np.nan
streak = 0

for i in tqdm(range(len(data_spy))):
  data_spy['Streak'][i] = streak
  if (data_spy['Close'][i] - data_spy['Open'][i]) > 0:
    data_spy['Green_day'][i] = True
    streak += 1
  else:
    streak = 0
    data_spy['Red_day'][i] = True

[*********************100%***********************]  1 of 1 completed


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



  0%|          | 0/252 [00:00<?, ?it/s]

In [ ]:
data_spy

,Open,High,Low,Close,Gap,Green_day,Red_day,Streak
Date,,,,,,,,
2020-09-25,322.579987,329.579987,321.640015,328.730011,NaN,1.0,NaN,0.0
2020-09-28,333.220001,334.959991,332.149994,334.190002,0.013659,1.0,NaN,1.0
2020-09-29,333.970001,334.769989,331.619995,332.369995,-0.000658,NaN,1.0,2.0
2020-09-30,333.089996,338.290009,332.880005,334.890015,0.002166,1.0,NaN,0.0
2020-10-01,337.690002,338.739990,335.010010,337.040009,0.008361,NaN,1.0,1.0
...,...,...,...,...,...,...,...,...
2021-09-20,434.880005,436.559998,428.859985,434.040009,-0.014771,NaN,1.0,0.0
2021-09-21,436.529999,437.910004,433.070007,433.630005,0.005737,NaN,1.0,0.0
2021-09-22,436.049988,440.029999,433.750000,437.859985,0.005581,1.0,NaN,0.0


In [ ]:
data_spy[data_spy['Streak'] >= 3]

In [ ]:
stock_data = PyTickerSymbols()

nasdaq_100=pd.DataFrame(stock_data.get_stocks_by_index('NASDAQ 100'))
nasdaq_100 = nasdaq_100[nasdaq_100.symbol != 'CTRP'].reset_index().symbol.to_list()

sp_500=pd.DataFrame(stock_data.get_stocks_by_index('S&P 500'))
sp_500 = sp_500[sp_500.symbol != 'SSE']
sp_500 = sp_500[sp_500.symbol != 'RTN']
sp_500 = sp_500[sp_500.symbol != 'CXO']
sp_500 = sp_500[sp_500.symbol != 'JEC']
sp_500 = sp_500[sp_500.symbol != 'SPY']
sp_500 = sp_500[sp_500.symbol != 'TIF'].reset_index().symbol.to_list()

stocks = nasdaq_100 + sp_500 #+ sp400

stocks = set(stocks) #Convierte la lista a Set, que lo único que hace es quitar elementos repetidos
stocks = list(stocks) #Reconvertir el set a list (para poder iterar sobre ella)

In [ ]:
#stocks = random.sample(stocks, 5)

In [ ]:
len(stocks)

517

In [ ]:
df_plot = pd.DataFrame()
for t in tqdm(range(len(stocks))):
  #OPCION PARA DATOS CON YFINANCE ::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
  data = yfinance.download(stocks[t], period='40d')#[['Open', 'High', 'Low', 'Close']]
  data['Ticker'] = stocks[t]
  data['Gap'] = np.nan
  data['prev_close'] = np.nan
  
  streak = 1
  for i in range(1,len(data)):

    gap = (data['Open'][i] - data['Close'][i-1]) / data['Close'][i-1]
    data['Gap'][i] = gap
    data['prev_close'][i] = data['Close'][i-1]

  df_plot = pd.concat([df_plot,data])

In [ ]:
df_plot

,Open,High,Low,Close,Adj Close,Volume,Ticker,Gap,prev_close
Date,,,,,,,,,
2021-07-30,78.730003,79.470001,76.410004,76.870003,76.870003,5976800.0,RCL,NaN,NaN
2021-08-02,76.970001,79.330002,75.080002,75.480003,75.480003,5211900.0,RCL,0.001301,76.870003
2021-08-03,75.099998,75.169998,71.790001,74.489998,74.489998,6002300.0,RCL,-0.005035,75.480003
2021-08-04,73.150002,74.230003,70.739998,72.650002,72.650002,7719100.0,RCL,-0.017989,74.489998
2021-08-05,72.599998,78.430000,72.570000,78.099998,78.099998,6755800.0,RCL,-0.000688,72.650002
...,...,...,...,...,...,...,...,...,...
2021-08-30,208.449997,209.410004,204.889999,204.990005,204.990005,398610000.0,LOW,0.012385,205.899994
2021-08-31,206.369995,206.589996,203.330002,203.889999,203.889999,487780000.0,LOW,0.006732,204.990005
2021-09-01,204.550003,205.039993,201.550003,203.429993,203.429993,283500000.0,LOW,0.003237,203.889999


In [ ]:
filtro = df_plot[((df_plot['Gap'] < -.02) | (df_plot['Gap'] > .02)) & (df_plot['Open'] > 10) & (df_plot['Volume'] > 750000)]

filtro['Spy_Gap'] = np.nan
filtro['Cerró'] = np.nan
filtro['Spy_Green'] = np.nan
filtro['Spy_Red'] = np.nan
filtro['Spy_Streak'] = np.nan

for i in range(len(filtro)):
  filtro['Spy_Gap'][i] = data_spy.loc[filtro.index[i].strftime('%Y-%m-%d')]['Gap']
  filtro['Spy_Green'][i] = data_spy.loc[filtro.index[i].strftime('%Y-%m-%d')]['Green_day']
  filtro['Spy_Red'][i] = data_spy.loc[filtro.index[i].strftime('%Y-%m-%d')]['Red_day']
  filtro['Spy_Streak'][i] = data_spy.loc[filtro.index[i].strftime('%Y-%m-%d')]['Streak']

  if filtro['Gap'][i] < 0 and filtro['High'][i] >= filtro['prev_close'][i]:
    filtro['Cerró'][i] = True
  elif filtro['Gap'][i] > 0 and filtro['Low'][i] <= filtro['prev_close'][i]:
    filtro['Cerró'][i] = True

In [ ]:
len(filtro)

440

In [ ]:
filtro[['Ticker', 'Gap','Spy_Gap', 'Spy_Green', 'Spy_Red', 'Spy_Streak', 'Cerró']][filtro['Cerró']>0]

# VERSION DEL 31/05/2021: ANALISIS COMPLETO (TODO UN AÑO) CON API

In [ ]:
stock_data = PyTickerSymbols()

nasdaq_100=pd.DataFrame(stock_data.get_stocks_by_index('NASDAQ 100'))
nasdaq_100 = nasdaq_100[nasdaq_100.symbol != 'CTRP'].reset_index().symbol.to_list()

sp_500=pd.DataFrame(stock_data.get_stocks_by_index('S&P 500'))
sp_500 = sp_500[sp_500.symbol != 'SSE']
sp_500 = sp_500[sp_500.symbol != 'RTN']
sp_500 = sp_500[sp_500.symbol != 'CXO']
sp_500 = sp_500[sp_500.symbol != 'JEC']
sp_500 = sp_500[sp_500.symbol != 'SPY']
sp_500 = sp_500[sp_500.symbol != 'TIF'].reset_index().symbol.to_list()

stocks = nasdaq_100 + sp_500 #+ sp400

stocks = set(stocks) #Convierte la lista a Set, que lo único que hace es quitar elementos repetidos
stocks = list(stocks) #Reconvertir el set a list (para poder iterar sobre ella)

In [ ]:
#stocks = random.sample(stocks, 100)

In [ ]:
len(stocks)

In [ ]:
df_plot = pd.DataFrame()
for t in tqdm(range(len(stocks))):
  #OPCION PARA DATOS CON YFINANCE ::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
  data = yfinance.download(stocks[t], period='10Y')#[['Open', 'High', 'Low', 'Close']]
  data['Ticker'] = stocks[t]
  data['Gap'] = np.nan
  data['prev_close'] = np.nan

  for i in range(1,len(data)):

    gap = (data['Open'][i] - data['Close'][i-1]) / data['Close'][i-1]
    data['Gap'][i] = gap
    data['prev_close'][i] = data['Close'][i-1]

  df_plot = pd.concat([df_plot,data])

In [ ]:
df_plot

In [ ]:
filtro = df_plot[(df_plot['Gap'] < -.02) & (df_plot['Open'] > 10) & (df_plot['Volume'] > 750000) & 
                 (df_plot['Gap'] > -.04)]

In [ ]:
risk = 1

filtro['Closed'] = np.nan
filtro['Green'] = np.nan
filtro['Red'] = np.nan
filtro['Stopped'] = np.nan

for i in range(len(filtro)):
  stop = filtro['Open'][i] - ((filtro['prev_close'][i] - filtro['Open'][i])/risk)
  if filtro['High'][i] >= filtro['prev_close'][i]:
    filtro['Closed'][i] = True
  if filtro['Low'][i] <= stop:
    filtro['Stopped'][i] = True

  if filtro['Closed'][i] != True and filtro['Stopped'][i] != True:
    if filtro['Close'][i] >= filtro['Open'][i]:
      filtro['Green'][i] = True
    else:
      filtro['Red'][i] = True

In [ ]:
filtro

In [ ]:
len(filtro)

In [ ]:
print(filtro['Closed'].count())
print(filtro['Green'].count())
print(filtro['Red'].count())
print(filtro['Stopped'].count())

In [ ]:
for i in tqdm(range(len(filtro))):

  example = yfinance.download(filtro.Ticker[i], start=(filtro.index[i]- timedelta(days=1)).strftime('%Y-%m-%d'), # 
                            end=(filtro.index[i] + timedelta(days=1)).strftime('%Y-%m-%d'), interval='2m')[['Open', 'High', 'Low', 'Close']]
  #Entry si el segundo candle cae debajo del primero
  # if len(example) > 0 and example.iloc[1]['Low'] < example.iloc[0]['Low']:
  #   stop = max(example.iloc[1]['High'], example.iloc[1]['High']) + .01 #Stop = valor máximo de los Highs del primer y segundo candle
  #   entry = example.iloc[0]['Low'] - .01 #Entry = Low del primer candle
  #   target = entry - (risk * (stop-entry)) #Target = el número de risks por el monto al stop
  #   for j in range(1,len(example)):
  #     if example.iloc[j]['High'] > stop:
  #       red_stocks.append(filtro.Ticker[i])
  #       break
  #     elif example.iloc[j]['Low'] < target:
  #       green_stocks.append(filtro.Ticker[i])
  #       break

  fig = go.Figure(data=[go.Candlestick(open=example.Open, high=example.High,low=example.Low, close=example.Close)])
  fig.update_layout(xaxis_rangeslider_visible=False)
  fig.update_layout(title=filtro.Ticker[i] + ' ' + filtro.index[i].strftime('%Y-%m-%d') + ' ' + str(filtro.Gap[i]))
  fig.show()

# CHEQUEO DIARIO, QUE ACCIONES HICIERON GAP?

In [8]:
stock_data = PyTickerSymbols()

nasdaq_100=pd.DataFrame(stock_data.get_stocks_by_index('NASDAQ 100'))
nasdaq_100 = nasdaq_100[nasdaq_100.symbol != 'CTRP'].reset_index().symbol.to_list()

sp_500=pd.DataFrame(stock_data.get_stocks_by_index('S&P 500'))
sp_500 = sp_500[sp_500.symbol != 'SSE']
sp_500 = sp_500[sp_500.symbol != 'RTN']
sp_500 = sp_500[sp_500.symbol != 'CXO']
sp_500 = sp_500[sp_500.symbol != 'JEC']
sp_500 = sp_500[sp_500.symbol != 'TIF'].reset_index().symbol.to_list()

total_tickers = nasdaq_100 + sp_500 #+ sp400

total_tickers = set(total_tickers) #Convierte la lista a Set, que lo único que hace es quitar elementos repetidos
total_tickers = list(total_tickers) #Reconvertir el set a list (para poder iterar sobre ella)

today = datetime.today().strftime('%Y-%m-%d')

In [ ]:
#total_tickers = random.sample(total_tickers, 5)

In [9]:
len(total_tickers)

516

In [14]:
#FILTRO PARA TICKERS
df_plot = pd.DataFrame()
#tickers_list = []
for i in tqdm(range(len(total_tickers))):
  ticker = yfinance.download(total_tickers[i], period='3D')#[['Close', 'Volume']]
  ticker['Ticker'] = total_tickers[i]
  ticker['prev_close'] = ticker['Close'].shift(1)
  ticker['Gap'] = (ticker['Open'] - ticker['Close'].shift(1)) / ticker['Close'].shift(1)

  df_plot = pd.concat([df_plot,ticker])

  0%|          | 0/516 [00:00<?, ?it/s]

[*********************100%***********************]  1 of 1 completed

1 Failed download:
- CBRE: No data found for this date range, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[****************

In [17]:
df_plot

,Open,High,Low,Close,Adj Close,Volume,Ticker,prev_close,Gap
Date,,,,,,,,,
2021-09-24,43.820000,44.660000,43.650002,44.099998,44.099998,2329000.0,VNO,NaN,NaN
2021-09-27,44.450001,45.320000,44.320000,44.369999,44.369999,1927000.0,VNO,44.099998,0.007937
2021-09-28,44.630001,45.139999,44.389999,44.820000,44.820000,104195.0,VNO,44.369999,0.005860
2021-09-24,40.240002,40.959999,39.950001,39.980000,39.980000,7087700.0,KR,NaN,NaN
2021-09-27,40.000000,41.049999,39.970001,40.189999,40.189999,7590600.0,KR,39.980000,0.000500
...,...,...,...,...,...,...,...,...,...
2021-09-27,71.389999,73.300003,71.389999,72.709999,72.709999,1434300.0,TXT,71.080002,0.004361
2021-09-28,72.779999,73.379997,72.099998,72.120003,72.120003,58723.0,TXT,72.709999,0.000963
2021-09-24,358.640015,361.690002,355.920013,361.549988,361.549988,180200.0,ANSS,NaN,NaN


In [18]:
df_plot[(df_plot['Gap'] < -.03) | (df_plot['Gap'] > .03)].loc[today]

,Open,High,Low,Close,Adj Close,Volume,Ticker,prev_close,Gap
Date,,,,,,,,,
2021-09-28,596.059998,606.359985,596.059998,599.900024,599.900024,344091.0,LRCX,616.669983,-0.033421
2021-09-28,14.630000,14.730000,14.390000,14.405000,14.405000,30724496.0,F,14.160000,0.033192
2021-09-28,93.080002,93.839897,91.000000,91.139999,91.139999,2263007.0,PDD,96.320000,-0.033638


In [ ]:
spy = yfinance.download('SPY', period='3D')#[['Close', 'Volume']]
spy['prev_close'] = spy['Close'].shift(1)
spy['Gap'] = (spy['Open'] - spy['Close'].shift(1)) / spy['Close'].shift(1)
spy